In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import style
import tables
import copy
import yaml

# the config file holds paths to the hdf5 files (in this example)
cfg_file = 'config.yaml'
cfg_file = yaml.safe_load(open(cfg_file))

In [ ]:
# want an E^-2 flux
def power_law_flux(e_in_gev):
    return 1E-8 * (e_in_gev**-2.)

livetime = 1 # make it Hz

### How to harvest weights from juliet files


In [ ]:
import weights_helper as weighting

# empty arrays to hold weights and charge
ehe_weights = np.asarray([])
ehe_energy_weights = None # for plotting in true space

energy_bins, energy_bin_centers = weighting.get_juliet_enu_binning()

In [ ]:

# juliet comes in five injected species, and two injected levels
# have to loop over all of them
juliet_species = ["nue", "numu", "nutau", "mu", "tau"]
juliet_energy_levels = ["high_energy", "very_high_energy"]
juliet_species = ['nue']
juliet_energy_levels = ["high_energy"]

# loop over all Juliet species
for s in juliet_species:
    for l in juliet_energy_levels:
            
        print(f"Working on juliet {s}, {l}")
        the_f = tables.open_file(cfg_file['juliet'][s][l]['file'])

        # get weighting information
        weight_dict, prop_matrix, evts_per_file = weighting.get_juliet_weightdict_and_propmatrix(the_f)
        evts_per_file = weighting.correct_events_per_file(s, l)
        n_gen = cfg_file['juliet'][s][l]['n_files'] * evts_per_file
    
        # individual event rates for this flux assumption
        weights = weighting.calc_juliet_weight_from_weight_dict_and_prop_matrix(
            weight_dict=weight_dict, prop_matrix=prop_matrix, 
            flux=power_law_flux, n_gen=n_gen, livetime=livetime
        )
        ehe_weights = np.concatenate((ehe_weights, weights))
        
        # total contribution to overall event rate in true energy space
        the_enu_weights = weighting.calc_juliet_flux_weight(
            weight_dict = weight_dict, prop_matrix = prop_matrix, 
            flux= power_law_flux, n_gen = n_gen, livetime=livetime,
        )
        if ehe_energy_weights is None:
            ehe_energy_weights = copy.deepcopy(the_enu_weights)
        else:
            ehe_energy_weights += copy.deepcopy(the_enu_weights)

        the_f.close()

In [ ]:
# save
np.savez('oneweight_information.npz',
         energy_bin_centers=energy_bin_centers,
         energy_bins=energy_bins,
         ehe_energy_weights=ehe_energy_weights,
         propagated_weights=ehe_weights
         )

### Event rate in true neutrino energy

In [ ]:
fig2, ax2 = plt.subplots(1, 1 , figsize=(7,5))
ax2.hist(energy_bin_centers, bins=energy_bins,
        weights=ehe_energy_weights, histtype='step', linewidth=3,
        density=True
        )
ax2.set_ylabel("PDF".format(livetime/60/60/24))
ax2.set_xlabel(r'Neutrino Energy [GeV]')
ax2.set_xscale('log')
ax2.set_yscale('log')